<div style='text-align: center;'>
    <img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQzJzIHdangJTrH2mFXFgsLjuLCjpfXXwbxg&usqp=CAU' width='100'/>
    <h1>Sharif University of Technology</h1>
    <h2>Natural Language Processing</h2>
    <h3>Final Project</h3>
    <h4>Spoiler classification and summary generation</h4>
    <p><strong>Authors:</strong> Ali Nikkhah, Ramtin Khoshnevis, Sarina Zahedi</p>
    <p><strong>(Equal Contribution)</strong></p>
</div>
<hr/>


In [ ]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
import string

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')


--------------------------------------------------------------------------------------------------------------------------------------


### **1. Load the Dataset**


In [ ]:
import pandas as pd
import json
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
file_path = '/kaggle/input/imdb-spoiler-dataset/IMDB_reviews.json'
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

df = pd.DataFrame(data)


### **2. Exploratory Data Analysis (EDA)**

In [ ]:
from tabulate import tabulate
import numpy as np

# Basic info
info = df.info()



In [ ]:
# Describe the dataset
description = df.describe()


# Check for missing values
missing_values = df.isnull().sum()


# Distribution of spoiler vs. non-spoiler
spoiler_distribution = df['is_spoiler'].value_counts(normalize=True)

# Length of reviews
df['review_length'] = df['review_text'].apply(len)
review_length_description = df['review_length'].describe()



In [ ]:
# Display the results

print("\nMissing Values:")
print(tabulate(missing_values.items(), headers=["Column", "Missing Values"], tablefmt="pretty"))

print("\nSpoiler vs. Non-Spoiler Distribution:")
print(tabulate(spoiler_distribution.items(), headers=["Is Spoiler", "Proportion"], tablefmt="pretty"))

print("\nReview Length Description:")
print(tabulate(review_length_description.items(), headers=["Statistic", "Value"], tablefmt="pretty"))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot some samples of the dataset
sample_size = 2

spoiler_samples = df[df['is_spoiler'] == True].sample(sample_size)
non_spoiler_samples = df[df['is_spoiler'] == False].sample(sample_size)

plt.figure(figsize=(40, 20))

# Spoiler samples
for i, review in enumerate(spoiler_samples['review_text']):
    plt.text(0.5, 1.0 - i*0.2, f"Spoiler Review {i+1}: {review[:150]}...", ha='center', va='top', wrap=True)

# Non-Spoiler samples
for i, review in enumerate(non_spoiler_samples['review_text']):
    plt.text(0.5, 0.5 - i*0.2, f"Non-Spoiler Review {i+1}: {review[:150]}...", ha='center', va='top', wrap=True)

plt.axis('off')
plt.title('Sample Reviews (Spoiler vs Non-Spoiler)')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot distribution of spoilers vs. non-spoilers
plt.figure(figsize=(8, 6))
sns.countplot(x='is_spoiler', data=df, palette='coolwarm')
plt.title('Distribution of Spoiler vs. Non-Spoiler Reviews')
plt.xlabel('Is Spoiler')
plt.ylabel('Count')
plt.show()

# Plot the distribution of review lengths
plt.figure(figsize=(10, 6))
sns.histplot(df['review_length'], kde=True, bins=30, color='purple')
plt.title('Distribution of Review Lengths')
plt.xlabel('Review Length')
plt.ylabel('Frequency')
plt.show()

# Correlation between review length and is_spoiler
plt.figure(figsize=(8, 6))
sns.boxplot(x='is_spoiler', y='review_length', data=df, palette='coolwarm')
plt.title('Review Length vs. Spoiler')
plt.xlabel('Is Spoiler')
plt.ylabel('Review Length')
plt.show()


### **2.B. Down sample to make two calsses uniform**

In [ ]:
class_counts = df['is_spoiler'].value_counts()
min_class_count = class_counts.min()

# Downsample each class to have the same number of instances as the smallest class
df_balanced = pd.DataFrame()  # Create an empty DataFrame to store the balanced data

for label in class_counts.index:
    df_balanced = pd.concat([
        df_balanced,
        df[df['is_spoiler'] == label].sample(n=min_class_count, random_state=42)
    ])

# Shuffle the DataFrame to mix the classes well
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df=df_balanced

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot distribution of spoilers vs. non-spoilers
plt.figure(figsize=(8, 6))
sns.countplot(x='is_spoiler', data=df, palette='coolwarm')
plt.title('Distribution of Spoiler vs. Non-Spoiler Reviews')
plt.xlabel('Is Spoiler')
plt.ylabel('Count')
plt.show()

# Plot the distribution of review lengths
plt.figure(figsize=(10, 6))
sns.histplot(df['review_length'], kde=True, bins=30, color='purple')
plt.title('Distribution of Review Lengths')
plt.xlabel('Review Length')
plt.ylabel('Frequency')
plt.show()

# Correlation between review length and is_spoiler
plt.figure(figsize=(8, 6))
sns.boxplot(x='is_spoiler', y='review_length', data=df, palette='coolwarm')
plt.title('Review Length vs. Spoiler')
plt.xlabel('Is Spoiler')
plt.ylabel('Review Length')
plt.show()


### **3. Data Preprocessing**

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm.notebook import tqdm
# Set of English stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    
    # Remove emails
    text = re.sub(r'\S*@\S*\s?', '', text)
    
    # Remove all non-word characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Normalize whitespaces
    text = re.sub(r'\s+', ' ', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    return text
# Set up tqdm for pandas apply
tqdm.pandas(desc="Cleaning Text")

# Apply the cleaning function with a progress bar
df['cleaned_review_text'] = df['review_text'].progress_apply(clean_text)

In [ ]:
# Specify the path where you want to save the JSON file
json_file_path = '/kaggle/working/preprocessed.json'  

# Save the DataFrame to a JSON file
df.to_json(json_file_path, orient='records', lines=True)

print(f"DataFrame saved successfully to {json_file_path}.")

In [ ]:
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer

# Encode the target label
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['is_spoiler'])
train_df, temp_df = train_test_split(df, test_size=0.8, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.05, random_state=42, stratify=temp_df['label'])

In [ ]:
class SpoilerDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512, stride=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.stride = stride

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer(
            text,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            stride=self.stride,
            return_overflowing_tokens=True
        )

        # Here you might want to handle multiple chunks if they exist
        if 'overflowing_tokens' in inputs:
            num_chunks = len(inputs['input_ids'])
            chunks = [{
                'input_ids': inputs['input_ids'][i].flatten(),
                'attention_mask': inputs['attention_mask'][i].flatten(),
                'label': torch.tensor(label, dtype=torch.long)
            } for i in range(num_chunks)]
            return chunks
        else:
            return {
                'input_ids': inputs['input_ids'].flatten(),
                'attention_mask': inputs['attention_mask'].flatten(),
                'label': torch.tensor(label, dtype=torch.long)
            }

In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create datasets with the resampled training data
train_dataset = SpoilerDataset(train_df['cleaned_review_text'].tolist(), train_df['label'].tolist(), tokenizer)
val_dataset = SpoilerDataset(val_df['cleaned_review_text'].tolist(), val_df['label'].tolist(), tokenizer)
test_dataset = SpoilerDataset(test_df['cleaned_review_text'].tolist(), test_df['label'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


### **4. Modeling**

In [ ]:
from transformers import BertForSequenceClassification
import torch.nn as nn
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
bert_model = bert_model.to(device)


# Define the loss function
loss_fn = nn.CrossEntropyLoss()


# Define optimizer (assuming the optimizer setup is missing in the provided code snippet)
optimizer = torch.optim.Adam(bert_model.parameters(), lr=2e-5)

### **5. Training and Evaluation**

BERT model

In [ ]:

import matplotlib.pyplot as plt

def train(model, dataloader, optimizer, loss_fn):
    model.train()
    total_loss, total_acc = 0, 0

    progress_bar = tqdm(dataloader, desc="Training")
    for batch in progress_bar:
        # Check if the batch is a list (multiple chunks)
        if isinstance(batch, list):
            # Process each chunk individually
            for chunk in batch:
                loss, acc = process_chunk(model, chunk, optimizer, loss_fn)
                total_loss += loss
                total_acc += acc
        else:
            # Process the batch normally if it's not a list
            loss, acc = process_chunk(model, batch, optimizer, loss_fn)
            total_loss += loss
            total_acc += acc

        progress_bar.set_postfix(loss=total_loss / len(dataloader), acc=total_acc / len(dataloader))

    return total_loss / len(dataloader), total_acc / len(dataloader)

def process_chunk(model, chunk, optimizer, loss_fn):
    optimizer.zero_grad()
    input_ids = chunk['input_ids'].to(device)
    attention_mask = chunk['attention_mask'].to(device)
    labels = chunk['label'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    preds = torch.argmax(outputs.logits, dim=1)
    acc = accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

    loss.backward()
    optimizer.step()

    return loss.item(), acc

def evaluate(model, dataloader):
    model.eval()
    total_acc = 0
    progress_bar = tqdm(dataloader, desc="Evaluating")

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            acc = accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

        total_acc += acc
        progress_bar.set_postfix(acc=total_acc / len(dataloader))

    return total_acc / len(dataloader)



In [ ]:
train_losses = []
train_accuracies = []
val_accuracies = []

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader  # Assuming you're using DataLoader
from tqdm.notebook import tqdm  # For progress bars in Jupyter Notebooks
import matplotlib.pyplot as plt  # For plotting graphs
from sklearn.metrics import accuracy_score


n_epochs = 15
for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    
    train_loss, train_acc = train(bert_model, train_loader, optimizer, loss_fn)

    # Append metrics
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    print(f'Epoch {epoch+1}: Train Loss {train_loss:.4f}')




In [ ]:
torch.save(bert_model, 'bert_model_complete.pth')
